<a href="https://colab.research.google.com/github/Churchiill/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# Import data

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Convert Dat. files to a CSV Function**


In [53]:
import pandas as pd
from pathlib import Path
import csv
from pathlib import Path

f = open("/content/draft.csv", "x")

def data2csv(path):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]
    n = Path(file).stem
    
    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['t',n,'e']) 
    df = df[n]
    if i==1:
      data = pd.concat([data,df], axis=1)
    else:
      data = df

    i = 1

  return data

# Converting data

# Anomalous-Cepheids

In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/phot.tar.gz').extractall('/content/drive/MyDrive')

In [58]:
LMC_ACEP_I = data2csv('/content/drive/MyDrive/phot/I')
LMC_ACEP_V = data2csv('/content/drive/MyDrive/phot/V')